In [29]:
import pickle
import pandas as pd
import numpy as np

In [30]:
vccs_prerequisite_dict = pickle.load(open("C:/Users/ys8mz/Box Sync/GAA Transfer/data/intermediate_files/ys8mz/degree_audit_analyses/vccs_prerequisite_dict_2021.p", "rb"))
# vccs_prerequisite_dict['MTH_263'] = [['MTH_167'], ['MTH_161', 'MTH_162'], ['MTH_163', 'MTH_164']]
# vccs_prerequisite_dict['EGR_126'] = [['CSC_110']]

In [31]:
all_vccsid_strm_1 = pd.read_stata("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\bias\\all_observations.dta").loc[:,['vccsid','college','strm','course','section']].drop_duplicates()
all_vccsid_strm_2 = pd.read_stata("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\bias\\all_vccsid_strm.dta")
all_observations = all_vccsid_strm_1.merge(all_vccsid_strm_2, on=['vccsid', 'strm'], how="inner")
all_observations.shape

(5168903, 6)

In [32]:
all_observations.loc[:,['vccsid', 'college', 'strm', 'course']].drop_duplicates().shape

(5164109, 4)

In [33]:
prior_courses = pd.read_stata("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\bias\\prior_courses.dta")
prior_courses = prior_courses.merge(all_observations.loc[:,['vccsid']].drop_duplicates(), how='inner', on=['vccsid'])
print(prior_courses.shape)
target_courses = pd.read_stata("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\bias\\part1_new.dta")
target_courses = target_courses.merge(all_observations, how='inner', on=['vccsid','college','strm','course','section'])
print(target_courses.shape)

(8069415, 5)
(5168903, 10)


In [34]:
def create_cdict(df):
    cdict = {}
    for j in range(df.shape[0]):
        course = df.course.iloc[j]
        credit = df.credit.iloc[j]
        est_grade = df.est_grade.iloc[j]
        try:
            cdict[course].append((credit,est_grade))
        except KeyError:
            cdict[course] = [(credit,est_grade)]
    return cdict.copy()

def calc_avg_grade(t):
    tot_cred = 0
    tot_grade_point = 0
    for p in t:
        tot_cred += p[0]
        tot_grade_point += p[0]*p[1]
    return tot_grade_point/tot_cred

In [35]:
vccsid_sets = []
l = list(np.unique(target_courses.vccsid))
n = len(l) // 500 + 1
for i in range(500):
    vccsid_sets.append(l[(n*i):(n*(i+1))])

In [18]:
for i,ll in enumerate(vccsid_sets):
    print(i)
    ll_series = pd.DataFrame({'vccsid':ll})
    prior_part = ll_series.merge(prior_courses, how='inner', on=['vccsid'])
    target_part = ll_series.merge(target_courses, how='inner', on=['vccsid'])
    print(prior_part.shape[0], target_part.shape[0])
    prior_part.to_stata("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\bias\\prior_courses\\part{}.dta".format(i),
                        write_index=False)
    target_part.to_stata("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\bias\\target_courses\\part{}.dta".format(i),
                        write_index=False)
    print("")

0
16841 10889

1
16427 10883

2
16488 10786

3
15802 10276

4
16180 10247

5
16646 10361

6
17135 10718

7
15978 10620

8
15626 10258

9
16129 10055

10
15711 10270

11
16642 10591

12
15684 9954

13
16130 10227

14
15871 10317

15
16280 10469

16
16415 10740

17
16477 10313

18
15743 10189

19
10900 8213

20
16921 10483

21
17210 10912

22
17030 10510

23
16526 10338

24
16506 10201

25
16185 10494

26
16825 10417

27
16383 10185

28
16784 10441

29
16277 10301

30
16640 10579

31
16426 10477

32
15547 10241

33
16453 10656

34
16075 10370

35
16370 10423

36
15900 10473

37
16727 10635

38
16606 10449

39
16156 10352

40
16035 10466

41
16106 10235

42
15813 10199

43
15783 10534

44
15862 10344

45
16455 10908

46
16116 10291

47
16052 10461

48
16769 10491

49
15949 10437

50
12106 8921

51
15034 9946

52
15726 10242

53
16557 10325

54
17086 10838

55
16069 10562

56
16799 10531

57
16481 10506

58
16400 10371

59
16995 10022

60
16243 9918

61
16341 9975

62
16394 10391

63
15872

17345 10954

493
17166 10786

494
15975 10036

495
17193 11066

496
16328 9927

497
16472 10578

498
16357 10383

499
15534 9833



In [36]:
# has_prereq_grade = []
# prereq_grade = []
# for i in range(target_courses.shape[0]):
#     if i % 1000 == 0:
#         print(i)
#     vccsid = target_courses.vccsid.iloc[i]
#     strm = target_courses.strm.iloc[i]
#     course = target_courses.course.iloc[i]
#     prior_sub = prior_courses[prior_courses.vccsid == vccsid]
#     prior_sub = prior_sub[prior_sub.strm < strm]
#     if prior_sub.shape[0] == 0:
#         has_prereq_grade.append(0)
#         prereq_grade.append(0)
#     else:
#         if course not in vccs_prerequisite_dict:
#             has_prereq_grade.append(0)
#             prereq_grade.append(0)
#             continue
#         else:
#             prerequisite_l = []
#             for l in vccs_prerequisite_dict[course]:
#                 prerequisite_l.extend(l)
#             cdict = create_cdict(prior_sub)
#             clist = list(cdict.keys())
#             common_l = np.intersect1d(prerequisite_l, clist)
#             if len(common_l) == 0:
#                 has_prereq_grade.append(0)
#                 prereq_grade.append(0)
#             else:
#                 new_t = []
#                 for c in common_l:
#                     new_t.extend(cdict[c])
#                 has_prereq_grade.append(1)
#                 prereq_grade.append(calc_avg_grade(new_t))

In [37]:
prereq_r = []
for i in range(500):
    prereq_r.extend(pickle.load(open("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\bias\\prereq\\part{}.p".format(i), "rb")))

In [38]:
prereq_df = pd.DataFrame(prereq_r, columns = ['vccsid', 'strm', 'college', 'course', 'section', 'has_prereq_grade', 'prereq_grade'])

In [39]:
print(target_courses.shape[0])
target_courses = target_courses.merge(prereq_df, how='inner', on=['vccsid', 'strm', 'college', 'course', 'section'])
print(target_courses.shape[0])

5168903
5168903


In [40]:
# has_repeat_grade = []
# repeat_grade = []
# for i in range(target_courses.shape[0]):
#     if i % 1000 == 0:
#         print(i)
#     vccsid = target_courses.vccsid.iloc[i]
#     strm = target_courses.strm.iloc[i]
#     course = target_courses.course.iloc[i]
#     prior_sub = prior_courses[prior_courses.vccsid == vccsid]
#     prior_sub = prior_sub[prior_sub.strm < strm]
#     if prior_sub.shape[0] == 0:
#         has_repeat_grade.append(0)
#         repeat_grade.append(0)
#     else:
#         cdict = create_cdict(prior_sub)
#         if course not in cdict:
#             has_repeat_grade.append(0)
#             repeat_grade.append(0)
#         else:
#             has_repeat_grade.append(1)
#             repeat_grade.append(calc_avg_grade(cdict[course]))

In [41]:
repeat_r = []
for i in range(500):
    repeat_r.extend(pickle.load(open("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\bias\\repeat\\part{}.p".format(i), "rb")))

In [42]:
repeat_df = pd.DataFrame(repeat_r, columns = ['vccsid', 'strm', 'college', 'course', 'section', 'has_repeat_grade', 'repeat_grade'])

In [43]:
print(target_courses.shape[0])
target_courses = target_courses.merge(repeat_df, how='inner', on=['vccsid', 'strm', 'college', 'course', 'section'])
print(target_courses.shape[0])

5168903
5168903


In [44]:
# target_courses.loc[:,'has_repeat_grade'] = has_repeat_grade
# target_courses.loc[:,'repeat_grade'] = repeat_grade
# target_courses.loc[:,'has_prereq_grade'] = has_prereq_grade
# target_courses.loc[:,'prereq_grade'] = prereq_grade

In [45]:
avg_grades = pd.read_stata("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\bias\\avg_grades.dta")
avg_dict = {}
avg_dict_2 = {}
for j in range(avg_grades.shape[0]):
    course = avg_grades.course.iloc[j]
    strm = avg_grades.strm.iloc[j]
    college = avg_grades.college.iloc[j]
    avg_dict[(course,strm,college)] = avg_grades.avg_grade.iloc[j]
    avg_dict_2[(strm,college)] = avg_grades.avg_grade_2.iloc[j]

In [46]:
avg_g = []
for i in range(target_courses.shape[0]):
    course = target_courses.course.iloc[i]
    strm = target_courses.strm.iloc[i]
    college = target_courses.college.iloc[i]
    try:
        avg_g.append(avg_dict[(course,strm,college)])
    except KeyError:
        avg_g.append(avg_dict_2[(strm,college)])
target_courses.loc[:,'avg_g'] = avg_g

In [47]:
target_courses.loc[:,'summer_ind'] = target_courses.strm.apply(lambda x: int(x % 10 == 3))

In [48]:
target_courses.columns.values

array(['vccsid', 'college', 'section', 'strm', 'section_size', 'eve_ind',
       'online_ind', 'lvl2_ind', 'course', 'first_strm',
       'has_prereq_grade', 'prereq_grade', 'has_repeat_grade',
       'repeat_grade', 'avg_g', 'summer_ind'], dtype=object)

In [49]:
target_courses = target_courses.loc[:,['vccsid','strm','college','course','section',
                                       'summer_ind', 'section_size', 'eve_ind', 'online_ind', 'lvl2_ind',
                                       'has_repeat_grade', 'repeat_grade', 'has_prereq_grade', 'prereq_grade', 
                                       'avg_g']]
print(target_courses.shape)

(5168903, 15)


In [50]:
target_courses.to_csv("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\bias\\course_specific_predictors.csv", index=False)